In [1]:
%load_ext cython

In [3]:
%%cython -a

cimport cython
import numpy as np

ctypedef fused flt_t:
    float
    double
    double complex

    
def get_dtype(flt_t[:, ::1] arg):
    if flt_t == float:
        return np.float32
    elif flt_t == double:
        return np.float64
    else:
        return np.complex128


@cython.boundscheck(False)
def matmul(flt_t[:, ::1] a, flt_t[:, ::1] b):
    
    if a.shape[1] != b.shape[0]:
        raise ValueError("incompatible dimensions.")
    
    dtyp = get_dtype(a)   # note that specialization are the same
    cdef flt_t[:, ::1] c = np.zeros((a.shape[0], b.shape[1]), 
                                    dtype=dtyp)
    
    cdef:
        int N = a.shape[0], L = a.shape[1], M = b.shape[1]
        int i, j, k
    
    for i in range(N):
        for j in range(M):
            for k in range(L):
                c[i, j] = c[i, j] + a[i, k] * b[k, j]
    
    return np.asarray(c)

In [4]:
matmul(np.zeros((3, 3), dtype=float), np.zeros((3, 4), dtype=float))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [6]:
rndm = np.random.RandomState(1234)

N, M = 111, 22
a = rndm.uniform(size=(N, M))
b = rndm.uniform(size=(M, N))

c = a @ b

cc = matmul(a, b)

In [40]:
a.dtype

dtype('float64')

In [7]:
(c - cc).max()

0.0

In [8]:
%timeit matmul(a, b)

329 µs ± 4.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit a @ b

51 µs ± 8.62 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
%%cython -a --compile-args=-O3

cimport cython
import numpy as np

ctypedef fused flt_t:
    float
    double
    double complex

    
def get_dtype(flt_t[:, ::1] arg):
    if flt_t == float:
        return np.float32
    elif flt_t == double:
        return np.float64
    else:
        return np.complex128


@cython.boundscheck(False)
def matmul2(flt_t[:, ::1] a, flt_t[:, ::1] b):
    
    if a.shape[1] != b.shape[0]:
        raise ValueError("incompatible dimensions.")
        
    dtyp = get_dtype(a)   # note that specialization are the same
    cdef flt_t[:, ::1] c = np.zeros((a.shape[0], b.shape[1]), dtype=dtyp)
    
    cdef flt_t aik
    
    cdef int N = a.shape[0], L = a.shape[1], M = b.shape[1]
    cdef int i, j, k

    for k in range(L):
        for i in range(N):
            aik = a[i, k]
            for j in range(M):
                c[i, j] = c[i, j] + aik * b[k, j]
    
    return np.asarray(c)

In [85]:
cc2 = matmul2(a, b)
cc = matmul(a, b)

(cc - cc2).max()

0.0

In [11]:
rndm = np.random.RandomState(1234)

M, N = 1000, 1000

a = np.random.uniform(size=(M, N))
b = np.random.uniform(size=(N, M))

%timeit matmul2(a, b)
%timeit matmul(a, b)

967 ms ± 5.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.24 s ± 7.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
%%cython?

In [12]:
%timeit a@b

65.2 ms ± 3.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [93]:
b.strides

(8000, 8)

In [5]:
%%cython --cplus

from libcpp.vector cimport vector

cdef vector[double] vec
vec.push_back(42)

print(vec[0])

42.0


In [ ]:
%%cython -a --compile-args=-O3

In [11]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp

from cython.parallel cimport prange

cdef int x

with nogil:
    for x in prange(10):
        pass

In [ ]:
 %%cython?